### 토큰화

In [1]:
import pandas as pd
from transformers import BertTokenizer
import os

base_dir = os.getcwd() + "/"

base_dir

#base_dir = "/content/drive/MyDrive/bookend/dev/text-style-classify/basic/"

/home/hasha/anaconda3/envs/AI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/hasha/dev/text-style-classify/basic/'

In [2]:
"""from google.colab import drive
drive.mount('/content/drive')"""

"from google.colab import drive\ndrive.mount('/content/drive')"

In [3]:
df1 = pd.read_csv(base_dir + "data/inmoon.csv")
df2 = pd.read_csv(base_dir + "data/science.csv")
df3 = pd.read_csv(base_dir + "data/tech.csv")

In [4]:
df1 = df1[["Author", "Classify_1", "Classify_2", "src_sentence"]]
df2 = df2[["Author", "Classify_1", "Classify_2", "src_sentence"]]
df3 = df3[["Author", "Classify_1", "Classify_2", "src_sentence"]]

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def tokenizing_encode(sentence):
    encoding = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=64,
        padding="max_length",
        return_attention_mask=False,
        return_tensors='pt',
        truncation=True
    )
    tokens = encoding['input_ids'][0]
    token_list = tokenizer.convert_ids_to_tokens(tokens)
    return pd.Series([tokens, token_list])

In [18]:
df1[["encoded_tokens", "tokenized_sentence"]] = df1["src_sentence"].apply(tokenizing_encode)
df2[["encoded_tokens", "tokenized_sentence"]] = df2["src_sentence"].apply(tokenizing_encode)
df3[["encoded_tokens", "tokenized_sentence"]] = df3["src_sentence"].apply(tokenizing_encode)

In [21]:
df1.to_pickle(base_dir +  f'data/compact_inmoon.pkl')
df2.to_pickle(base_dir +  f'data/compact_tech.pkl')
df3.to_pickle(base_dir +  f'data/compact_science.pkl')

In [20]:
df= pd.concat([df1, df2, df3])
df = df.reset_index(drop=True, inplace=False)

df

,Author,Classify_1,Classify_2,src_sentence,"(encoded_tokens, tokenized_sentence)",encoded_tokens,tokenized_sentence
0,김명호,인문학,역사/고고학,항미원조 지원군이 관리하던 외국인 포로수용소와 판문점을 외교관 양성소로 활용했다.,"([tensor(101), tensor(9959), tensor(22458), te...","[tensor(101), tensor(9959), tensor(22458), ten...","[[CLS], 항, ##미, ##원, ##조, 지, ##원, ##군이, 관, ##리..."
1,김명호,인문학,역사/고고학,말장난 못지않게 글 장난도 시간 낭비라고 생각해서가 아니다.,"([tensor(101), tensor(9251), tensor(13890), te...","[tensor(101), tensor(9251), tensor(13890), ten...","[[CLS], 말, ##장, ##난, 못, ##지, ##않, ##게, 글, 장, #..."
2,김명호,인문학,역사/고고학,"일기건 편지건, 남들이 쓴 걸 보기만 했지 직접 써본 적이 거의 없기 때문이다.","([tensor(101), tensor(9641), tensor(12310), te...","[tensor(101), tensor(9641), tensor(12310), ten...","[[CLS], 일, ##기, ##건, 편, ##지, ##건, ,, 남, ##들이, ..."
3,김명호,인문학,역사/고고학,"20세기 이전 생몰의 인명, 잡지와 신문명, 좀 더 친숙하거나 뜻을 잘 드러내는 일...","([tensor(101), tensor(10197), tensor(47955), t...","[tensor(101), tensor(10197), tensor(47955), te...","[[CLS], 20, ##세기, 이전, 생, ##몰, ##의, 인, ##명, ,, ..."
4,김명호,인문학,역사/고고학,천안문 사태 이후 호주에 머물며 가끔 홍콩을 오간다며 웃었다.,"([tensor(101), tensor(9746), tensor(34951), te...","[tensor(101), tensor(9746), tensor(34951), ten...","[[CLS], 천, ##안, ##문, 사, ##태, 이후, 호, ##주, ##에, ..."
...,...,...,...,...,...,...,...
812492,김미선,기술과학,에너지/자원,"현재 일본은 이때 확보한 원천 기술을 바탕으로 2단계 연구를 기획하고 있으며, 국제...","([tensor(101), tensor(26565), tensor(23130), t...","[tensor(101), tensor(26565), tensor(23130), te...","[[CLS], 현재, 일본, ##은, 이때, 확, ##보, ##한, 원, ##천, ..."
812493,김미선,기술과학,에너지/자원,석유나 천연가스와 같은 화석 연료가 빈곤한 우리는 모든 에너지를 외국으로부터 수입해...,"([tensor(101), tensor(9426), tensor(42815), te...","[tensor(101), tensor(9426), tensor(42815), ten...","[[CLS], 석, ##유, ##나, 천, ##연, ##가, ##스, ##와, 같은..."
812494,김미선,기술과학,에너지/자원,"주 열량은 발열량 또는 연소열이라고도 부르는데, 액체 연료 1킬로그램이 온도 0℃,...","([tensor(101), tensor(9689), tensor(9569), ten...","[tensor(101), tensor(9689), tensor(9569), tens...","[[CLS], 주, 열, ##량, ##은, 발, ##열, ##량, 또는, 연, ##..."
812495,김미선,기술과학,에너지/자원,"이는 물질마다 가지고 있는 에너지가 모두 다르기 때문에 일어나는 현상인데, 액체 수...","([tensor(101), tensor(33904), tensor(9299), te...","[tensor(101), tensor(33904), tensor(9299), ten...","[[CLS], 이는, 물, ##질, ##마다, 가지고, 있는, 에, ##너, ##지..."


In [9]:
df.to_pickle(base_dir +  f'data/compact.pkl')

### author 인코딩

In [10]:
#df = pd.read_pickle(base_dir +  f'data/all.pkl')

In [22]:
author_set = set(df['Author'].to_list())

author_dict = {author : i for i, author in enumerate(author_set)}

import json

with open(base_dir + "data/author_encoding.json", "w") as json_file:
    json.dump(author_dict, json_file, indent=4)

df['encoded_author'] = df['Author'].apply(lambda x : author_dict[x])

### 장르 인코딩

In [23]:
#df = pd.read_pickle(base_dir +  f'data/all.pkl')

In [24]:
df['class'] = df['Classify_1']+"/" + df['Classify_2']

classify_set = set(df['class'].to_list())

classify_dict = {classs : i for i, classs in enumerate(classify_set)}

import json

with open(base_dir + "data/class_encoding.json", "w") as json_file:
    json.dump(classify_dict, json_file, indent=4)

df['encoded_class'] = df['class'].apply(lambda x : classify_dict[x])

In [25]:
df.to_pickle(base_dir +  f'data/labeled_compact.pkl')

In [26]:
df

,Author,Classify_1,Classify_2,src_sentence,"(encoded_tokens, tokenized_sentence)",encoded_tokens,tokenized_sentence,encoded_author,class,encoded_class
0,김명호,인문학,역사/고고학,항미원조 지원군이 관리하던 외국인 포로수용소와 판문점을 외교관 양성소로 활용했다.,"([tensor(101), tensor(9959), tensor(22458), te...","[tensor(101), tensor(9959), tensor(22458), ten...","[[CLS], 항, ##미, ##원, ##조, 지, ##원, ##군이, 관, ##리...",2117,인문학/역사/고고학,5
1,김명호,인문학,역사/고고학,말장난 못지않게 글 장난도 시간 낭비라고 생각해서가 아니다.,"([tensor(101), tensor(9251), tensor(13890), te...","[tensor(101), tensor(9251), tensor(13890), ten...","[[CLS], 말, ##장, ##난, 못, ##지, ##않, ##게, 글, 장, #...",2117,인문학/역사/고고학,5
2,김명호,인문학,역사/고고학,"일기건 편지건, 남들이 쓴 걸 보기만 했지 직접 써본 적이 거의 없기 때문이다.","([tensor(101), tensor(9641), tensor(12310), te...","[tensor(101), tensor(9641), tensor(12310), ten...","[[CLS], 일, ##기, ##건, 편, ##지, ##건, ,, 남, ##들이, ...",2117,인문학/역사/고고학,5
3,김명호,인문학,역사/고고학,"20세기 이전 생몰의 인명, 잡지와 신문명, 좀 더 친숙하거나 뜻을 잘 드러내는 일...","([tensor(101), tensor(10197), tensor(47955), t...","[tensor(101), tensor(10197), tensor(47955), te...","[[CLS], 20, ##세기, 이전, 생, ##몰, ##의, 인, ##명, ,, ...",2117,인문학/역사/고고학,5
4,김명호,인문학,역사/고고학,천안문 사태 이후 호주에 머물며 가끔 홍콩을 오간다며 웃었다.,"([tensor(101), tensor(9746), tensor(34951), te...","[tensor(101), tensor(9746), tensor(34951), ten...","[[CLS], 천, ##안, ##문, 사, ##태, 이후, 호, ##주, ##에, ...",2117,인문학/역사/고고학,5
...,...,...,...,...,...,...,...,...,...,...
812492,김미선,기술과학,에너지/자원,"현재 일본은 이때 확보한 원천 기술을 바탕으로 2단계 연구를 기획하고 있으며, 국제...","([tensor(101), tensor(26565), tensor(23130), t...","[tensor(101), tensor(26565), tensor(23130), te...","[[CLS], 현재, 일본, ##은, 이때, 확, ##보, ##한, 원, ##천, ...",1341,기술과학/에너지/자원,6
812493,김미선,기술과학,에너지/자원,석유나 천연가스와 같은 화석 연료가 빈곤한 우리는 모든 에너지를 외국으로부터 수입해...,"([tensor(101), tensor(9426), tensor(42815), te...","[tensor(101), tensor(9426), tensor(42815), ten...","[[CLS], 석, ##유, ##나, 천, ##연, ##가, ##스, ##와, 같은...",1341,기술과학/에너지/자원,6
812494,김미선,기술과학,에너지/자원,"주 열량은 발열량 또는 연소열이라고도 부르는데, 액체 연료 1킬로그램이 온도 0℃,...","([tensor(101), tensor(9689), tensor(9569), ten...","[tensor(101), tensor(9689), tensor(9569), tens...","[[CLS], 주, 열, ##량, ##은, 발, ##열, ##량, 또는, 연, ##...",1341,기술과학/에너지/자원,6
812495,김미선,기술과학,에너지/자원,"이는 물질마다 가지고 있는 에너지가 모두 다르기 때문에 일어나는 현상인데, 액체 수...","([tensor(101), tensor(33904), tensor(9299), te...","[tensor(101), tensor(33904), tensor(9299), ten...","[[CLS], 이는, 물, ##질, ##마다, 가지고, 있는, 에, ##너, ##지...",1341,기술과학/에너지/자원,6
